# Import

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd 
import numpy as np
import scipy.io
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import (accuracy_score, mean_squared_error)

Using TensorFlow backend.


# Loading Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import scipy.io
mat = scipy.io.loadmat('/content/drive/My Drive/Colab Notebooks/Machine Learning For Data Mining/MEG Data/SUB_100307_S2.mat')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data Preprocessing

In [3]:
data = mat['D']
print(data.shape)

X = np.reshape(data, (240, 2035*124))
X = np.transpose(X)
print(X.shape)


(240, 2035, 124)
(252340, 240)


# PCA And RMSE

In [0]:
def do_pca(input_data, variance):
  '''
  print("-----------------------------------------------------------------------------------------------------------------------")
  print()
  print("initial shape of data: ",input_data.shape)
  '''
  scaler = StandardScaler()
  input_data = scaler.fit_transform(input_data)

  pca = PCA(n_components = variance)
  principalComponents = pca.fit_transform(input_data)
  
  comp = pca.components_
  
  print(principalComponents.shape, comp.shape)
  reproject_data = np.matmul( principalComponents, comp)
  '''
  print("Shape of data after PCA: ",principalComponents.shape)

  print("PCA explained varience ratio: ",pca.explained_variance_ratio_)
  plt.plot(pca.explained_variance_ratio_)
  plt.title('PCA Explained Variance Ratio')
  plt.grid(True,which='major',axis='both', linestyle = '-')
  plt.grid(True,which='minor',axis='both', linestyle = '--')
  plt.minorticks_on()
  plt.show()

  print("-----------------------------------------------------------------------------------------------------------------------")
  '''
  return reproject_data

In [10]:
reproject_data = do_pca(X,0.95)

rmse = np.sqrt(mean_squared_error(X, reproject_data))
print('RMSE:', rmse)

(252340, 74) (74, 240)
RMSE: 0.9749698776778162


# Auto Encoder

In [11]:
input_dim = 240
output_dim = 240
bottlenect_dim = 74

autoencoder = Sequential([
        Dense((157), activation='linear',input_shape = (240,)),
        Dense(90, activation='linear'),
        Dense(74, activation='linear'),
        Dense(90, activation='linear'),
        Dense(157, activation='linear'),
        Dense(240, activation='linear'),
])

autoencoder.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mean_squared_error'])
autoencoder.fit(X, X, epochs = 30, batch_size = 256, verbose = 1)

X_pred = autoencoder.predict(X)
rmse = np.sqrt(mean_squared_error(X, X_pred))
print("Rmse: ",rmse)








Epoch 1/30





252340/252340 [==============================] - 12s 49us/step - loss: 6.9100e-08 - mean_squared_error: 6.9100e-08
Epoch 2/30
252340/252340 [==============================] - 7s 29us/step - loss: 2.6423e-08 - mean_squared_error: 2.6423e-08
Epoch 3/30
252340/252340 [==============================] - 7s 28us/step - loss: 2.7917e-08 - mean_squared_error: 2.7917e-08
Epoch 4/30
252340/252340 [==============================] - 7s 28us/step - loss: 2.5244e-08 - mean_squared_error: 2.5244e-08
Epoch 5/30
252340/252340 [==============================] - 7s 29us/step - loss: 2.4041e-08 - mean_squared_error: 2.4041e-08
Epoch 6/30
252340/252340 [==============================] - 7s 28us/step - loss: 2.2020e-08 - mean_squared_error: 2.2020e-08
Epoch 7/30
252340/252340 [==============================] - 7s 28us/step - loss: 2.1012e-08 - mean_squared_error: 2.1012e-08
Epoch 8/30
252340/252340 [==============================] - 7s 28us/step - loss: 2.0289e-08 - mean_squared_error